# Budget vs Execution Analysis (2019 & 2021)

Red Flag: All sectors execute 93–96% — unusually high. But is it real or just accounting?

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
# Load budget execution data
# Try different possible file paths
try:
    df = pd.read_csv("../data/processed/budget_execution_all_years.csv")
    print(f"Loaded budget execution data with {len(df)} rows")
except FileNotFoundError:
    print("Budget execution data file not found")
    # Create sample data for demonstration
    df = pd.DataFrame([
        {"year": 2019, "sector": "Administración", "budget": 250000000, "execution": 240000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Obras Públicas", "budget": 300000000, "execution": 285000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Desarrollo Social", "budget": 150000000, "execution": 144000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Salud", "budget": 180000000, "execution": 171000000, "Quarter": "Annual"},
        {"year": 2019, "sector": "Educación", "budget": 200000000, "execution": 192000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Administración", "budget": 280000000, "execution": 265000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Obras Públicas", "budget": 350000000, "execution": 330000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Desarrollo Social", "budget": 170000000, "execution": 162000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Salud", "budget": 210000000, "execution": 200000000, "Quarter": "Annual"},
        {"year": 2021, "sector": "Educación", "budget": 230000000, "execution": 218000000, "Quarter": "Annual"}
    ])
    print(f"Created sample budget execution data with {len(df)} rows")

In [ ]:
# Calculate underspent and execution rate
df["underspent"] = df["budget"] - df["execution"]
df["execution_rate"] = df["execution"] / df["budget"] * 100
df["execution_rate_formatted"] = df["execution_rate"].map('{:,.2f}%'.format)

In [ ]:
# Chart: Budget vs Execution for 2019
df_2019 = df[df["year"] == 2019]

fig = px.bar(df_2019, x="sector", y=["budget", "execution"],
             title="2019: ¿Presupuesto vs. Ejecución Real?",
             barmode="group", labels={"value": "ARS"},
             color_discrete_sequence=['#1f77b4', '#ff7f0e'])

# Add annotation about obras públicas
fig.add_annotation(x=1, y=310000000, text="¿Dónde están las obras?", showarrow=True, arrowhead=1)

# Update layout
fig.update_layout(
    yaxis_title="Monto (ARS)",
    xaxis_title="Sector",
    legend_title="Tipo"
)

# Show the chart
fig.show()

# Save to HTML
fig.write_html("../public/charts/budget_vs_execution_2019.html")

In [ ]:
# Chart: Budget vs Execution for 2021
df_2021 = df[df["year"] == 2021]

fig = px.bar(df_2021, x="sector", y=["budget", "execution"],
             title="2021: ¿Presupuesto vs. Ejecución Real?",
             barmode="group", labels={"value": "ARS"},
             color_discrete_sequence=['#2ca02c', '#d62728'])

# Update layout
fig.update_layout(
    yaxis_title="Monto (ARS)",
    xaxis_title="Sector",
    legend_title="Tipo"
)

# Show the chart
fig.show()

# Save to HTML
fig.write_html("../public/charts/budget_vs_execution_2021.html")

In [ ]:
# Combined chart for both years
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('2019: Presupuesto vs Ejecución', '2021: Presupuesto vs Ejecución'),
    shared_xaxes=True,
    vertical_spacing=0.1
)

# Add traces for 2019
df_2019 = df[df["year"] == 2019]
fig.add_trace(
    go.Bar(name='Presupuestado 2019', x=df_2019["sector"], y=df_2019["budget"],
           marker_color='lightblue'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='Ejecutado 2019', x=df_2019["sector"], y=df_2019["execution"],
           marker_color='orange'),
    row=1, col=1
)

# Add traces for 2021
df_2021 = df[df["year"] == 2021]
fig.add_trace(
    go.Bar(name='Presupuestado 2021', x=df_2021["sector"], y=df_2021["budget"],
           showlegend=False, marker_color='lightgreen'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(name='Ejecutado 2021', x=df_2021["sector"], y=df_2021["execution"],
           showlegend=False, marker_color='red'),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Comparación 2019 vs 2021: Presupuesto vs Ejecución",
    barmode="group",
    yaxis_title="Monto (ARS)",
    xaxis_title="Sector",
    legend_title="Tipo"
)

fig.show()
# Save to HTML
fig.write_html("../public/charts/budget_vs_execution_comparison.html")

In [ ]:
# Analysis of execution rates
execution_analysis = df.groupby(['year', 'sector']).agg({
    'budget': 'first',
    'execution': 'first',
    'execution_rate': 'first'
}).reset_index()

print("Execution Analysis by Year and Sector:")
print(execution_analysis)

# Check for unusually high execution rates (>95%)
high_execution = execution_analysis[execution_analysis['execution_rate'] > 95]
print(f"\nSectors with >95% execution rate:")
print(high_execution)

# Narrative text
print("\n📰 Narrative:")
print("'El municipio dice ejecutar el 95% del presupuesto… pero en 2019, $300M para Obras Públicas no se tradujeron en calles, cloacas ni viviendas visibles. ¿Se pagaron o solo se comprometieron?' ")

In [ ]:
# Summary statistics
stats = df.groupby('year').agg({
    'execution_rate': ['mean', 'min', 'max', 'std']
}).round(2)

print("\nSummary Statistics by Year:")
print(stats)